In [3]:
#pip install gym

In [10]:
#load packages
import gym
import numpy as np
from gym import spaces
from scipy.stats import norm

In [130]:
class invasive_IPM(gym.Env):
    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        params={"growth_k": 0.43, "growth_xinf": 109, "growth_sd": 2.5, "nmortality": 0.03,
               "trapm_sigma": 0.15, "trapm_xmax": 44, "trapm_pmax": 0.0005, "trapf_pmax": 0.0008,
               "trapf_k": 0.5, "trapf_midpoint": 45, "init_mean_recruit": 15, "init_sd_recruit": 1.5,
               "init_mean_adult": 65, "init_sd_adult": 8, "init_n_recruit": 1000, "init_n_adult": 1000,
               "w_mort_scale": 5, "K": 25000, "imm": 100, "r": 0.8, "area": 4000,"loss_a": 0.265,
               "loss_b": 2.80, "loss_c": 2.99, "minsize": 5, "maxsize": 110, "nsize": 21, "delta_t": 1/12},
        Tmax=100,
        file=None,
    ):
        
        # parameters
        self.growth_k = params["growth_k"]
        self.growth_xinf = params["growth_xinf"]
        self.growth_sd = params["growth_sd"]
        self.nmortality = params["nmortality"]
        self.trapm_sigma = params["trapm_sigma"]
        self.trapm_xmax = params["trapm_xmax"]
        self.trapm_pmax = params["trapm_pmax"]
        self.trapf_pmax = params["trapf_pmax"]
        self.trapf_k = params["trapf_k"]
        self.trapf_midpoint = params["trapf_midpoint"]
        self.init_mean_recruit = params["init_mean_recruit"]
        self.init_sd_recruit = params["init_sd_recruit"]
        self.init_mean_adult = params["init_mean_adult"]
        self.init_sd_adult = params["init_sd_adult"]
        self.init_n_recruit = params["init_n_recruit"]
        self.init_n_adult = params["init_n_adult"]
        self.w_mort_scale = params["w_mort_scale"]
        self.K = params["K"]
        self.imm = params["imm"]
        self.r = params["r"]
        self.area = params["area"]
        self.loss_a = params["loss_a"]
        self.loss_b = params["loss_b"]
        self.loss_c = params["loss_c"]
        self.minsize = params["minsize"]
        self.maxsize = params["maxsize"]
        self.nsize = params["nsize"]
        self.delta_t = params["delta_t"]
        self.params = params

        # Preserve these for reset
        self.observations = np.array([0,0,0,0,0,0,0,0,0], dtype=np.float32)
        self.reward = 0
        self.years_passed = 0
        self.Tmax = Tmax
                
        # Initial state
        self.state = init_state(self)

        # Action space
        self.action_space = spaces.Box(
            np.array([0], dtype=np.float32),
            np.array([1000], dtype=np.float32),
            dtype=np.float32,
        )
        
        # Observation space
        self.observation_space = spaces.Box(
            np.array([0,0,0,0,0,0,0,0,0], dtype=np.float32),
            np.array([2000,2000,2000,2000,2000,2000,2000,2000,2000], dtype=np.float32),
            dtype=np.float32,
        )
        
    def step(self,action):
        #size selective harvest rate, given action
        harvest_rate = 1-np.exp(-(size_sel_norm(self)*action + size_sel_log(self)*action))

        #add pop at t=1
        size_freq = np.zeros(shape=(self.nsize,9),dtype='object')
        size_freq[:,0] = self.state

        #create array to store # removed
        removed = np.zeros(shape=(self.nsize,9),dtype='object')
        
        #calculate removed and record observation at t=1
        #apply monthly harvest rate
        for k in range(21):
            removed[k,0] = np.random.binomial(size_freq[k,0], harvest_rate[k])
        #record the catch/effort in the observation space
        self.observations[0] = np.sum(removed[:,0])/(action*2)

        #loop through intra-annual change (9 total months), t=2+
        for j in range(8):
            #project to next month
            for k in range(21):
                #project to next size frequency
                size_freq[k,j+1] = np.random.binomial(n=(g_m_kernel(self)@(size_freq[:,j] - removed[:,j]))[k], 
                                                      p=np.exp(-self.nmortality))
                #apply monthly harvest rate
                removed[k,j+1] = np.random.binomial(size_freq[k,j+1], harvest_rate[k])
            #record the catch/effort in the observation space
            #### maybe remove effort
            self.observations[j+1] = np.sum(removed[:,j+1])/(action*2)

        #calculate new adult population after overwinter mortality
        new_adults = size_freq[:,8]*np.exp(-w_mortality(self))

        #simulate new recruits
        local_recruits = np.random.normal(dd_growth(self),dd_growth(self)/4)
        nonlocal_recruits = np.random.normal(self.imm,self.imm/4)*(1-np.sum(self.state)/self.K)
        recruit_total = local_recruits + nonlocal_recruits

        #get sizes of recruits
        recruit_sizes = (norm.cdf(boundary(self)[1:(self.nsize+1)],self.init_mean_recruit,self.init_sd_recruit)-\
         norm.cdf(boundary(self)[0:self.nsize],self.init_mean_recruit,self.init_sd_recruit))*recruit_total

        #store new population size
        self.state = recruit_sizes + new_adults

        #calculate reward
        self.reward = reward_func(self)
        self.years_passed += 1

        done = bool(self.years_passed > self.Tmax)

        if np.sum(self.state) <= 0.0:
            done = True

        return self.observations, self.reward, done, done, {}
        
    def reset(self):
        self.state = init_state(self)
        self.years_passed = 0

        # for tracking only
        self.reward = 0

        self.observations = np.array([0,0,0,0,0,0,0,0,0], dtype=np.float32)

        return self.observations

In [131]:
#################
#helper functions

#set up boundary points of IPM mesh
def boundary(self):
    boundary = self.minsize+np.arange(0,(self.nsize+1),1)*(self.maxsize-self.minsize)/self.nsize
    return boundary

#set up mid points of IPM mesh
def midpoints(self):
    midpoints = 0.5*(boundary(self)[0:self.nsize]+boundary(self)[1:(self.nsize+1)])
    return midpoints

#function for initial state
def init_state(self):
    init_pop = (norm.cdf(boundary(self)[1:(self.nsize+1)],self.init_mean_adult,self.init_sd_adult)-\
     norm.cdf(boundary(self)[0:self.nsize],self.init_mean_adult,self.init_sd_adult))*self.init_n_adult+\
    (norm.cdf(boundary(self)[1:(self.nsize+1)],self.init_mean_recruit,self.init_sd_recruit)-\
     norm.cdf(boundary(self)[0:self.nsize],self.init_mean_recruit,self.init_sd_recruit))*self.init_n_recruit
    return init_pop

#function for logistic size selectivity curve
def size_sel_log(self):
    size_sel = self.trapf_pmax/(1+np.exp(-self.trapf_k*(midpoints(self)-self.trapf_midpoint)))
    return size_sel

#function for gaussian size selectivity curve
def size_sel_norm(self):
    size_sel = self.trapm_pmax*np.exp(-(midpoints(self)-self.trapm_xmax)**2/2*self.trapm_sigma**2)
    return size_sel

#function for growth/mortality kernel
def g_m_kernel(self):
    array = np.empty(shape=(self.nsize,self.nsize),dtype='object')
    for i in range(self.nsize):
        mean = (self.growth_xinf-midpoints(self)[i])*(1-np.exp(-self.growth_k*self.delta_t)) + midpoints(self)[i]
        array[:,i] = (norm.cdf(boundary(self)[1:(self.nsize+1)],mean,self.growth_sd)-\
                      norm.cdf(boundary(self)[0:self.nsize],mean,self.growth_sd))
    return array

#function for overwinter mortality
def w_mortality(self):
    wmort = self.w_mort_scale/midpoints(self)
    return wmort

#function for density dependent growth
def dd_growth(self):
    dd_recruits = np.sum(self.state)*self.r*(1-np.sum(self.state)/self.K)
    return dd_recruits

#function for reward
def reward_func(self):
    reward = -self.loss_a/(1+np.exp(-self.loss_b*(np.sum(self.state)/self.area-self.loss_c)))
    return reward

In [132]:
#set env
env=invasive_IPM()

In [110]:
#calculate reward after one iteration, action = 25 
env.reset()
observations,reward,terminated,done,info=env.step(500)
reward

-0.00023878203828133656

In [133]:
#create empty array to store observations across all years
observations_matrix = np.empty(shape=(30,9),dtype='object')

#create empty vector to store reward
reward_vec = []

env.reset()
episode_reward = 0.0
for t in range(30):
    observations,reward,terminated,done,info=env.step(200)
    observations_matrix[t,:] = observations
    reward_vec = np.append(reward_vec,reward)
    episode_reward += reward

In [135]:
print(observations_matrix)

[[0.3824999928474426 0.27000001072883606 0.2549999952316284
  0.26499998569488525 0.21250000596046448 0.1899999976158142
  0.20250000059604645 0.18250000476837158 0.20749999582767487]
 [0.14749999344348907 0.14749999344348907 0.1875 0.17249999940395355
  0.14000000059604645 0.16249999403953552 0.13249999284744263
  0.18250000476837158 0.18000000715255737]
 [0.14749999344348907 0.15000000596046448 0.14000000059604645
  0.14249999821186066 0.20000000298023224 0.19249999523162842
  0.20749999582767487 0.27000001072883606 0.2750000059604645]
 [0.20499999821186066 0.21250000596046448 0.20749999582767487
  0.19249999523162842 0.20999999344348907 0.22499999403953552
  0.2824999988079071 0.26750001311302185 0.2574999928474426]
 [0.19499999284744263 0.26750001311302185 0.25 0.24250000715255737
  0.2175000011920929 0.22499999403953552 0.26249998807907104
  0.2574999928474426 0.27250000834465027]
 [0.2750000059604645 0.3174999952316284 0.25 0.2874999940395355
  0.23999999463558197 0.2725000083446

In [136]:
print(episode_reward)

-0.30891891049949727


In [23]:
#example: testing functions
#self = invasive_IPM()
#action = 25
#self.reset()
#-self.loss_a/(1+np.exp(self.loss_b*(np.sum(self.state)/self.area-self.loss_c)))